# Gaussian elimination - pivoting

The [Gaussian elimination](https://en.wikipedia.org/wiki/Gaussian_elimination) iteratively transforms an unstructured linear system $\mathbf{A}\mathbf{x} = \mathbf{y}$ into an equivalent triangular system $\mathbf{B}\mathbf{x} = \mathbf{z}$ having the same solution $\mathbf{x}$. As pointed out in the previous class, the **pivots** needed for computing the **Gauss multipliers** must be nonzero. The presence of small pivots produces arbitrarily poor results, even for well-conditioned problems, showing that Gauss elimination may be an unstable method, depending on the elements of $\mathbf{A}$ (Golub and Van Loan, 2013).

To avoid the division by small pivots and guarantee that the pivot has the largest absolute value, we may always swap the lines. 

Let's recall the linear system $\mathbf{A}\mathbf{x} = \mathbf{y}$ presented in the previous class:

In [1]:
import numpy as np
from scipy.linalg import lu

In [2]:
A = np.array([[2.,1.,-1.],
              [-3.,-1.,2.],
              [-2.,1.,2.]])

In [3]:
y = np.array([8., -11., -3.])

The solution of this system is given by:

In [4]:
x = np.linalg.solve(A,y)

In [5]:
print x

[ 2.  3. -1.]


This system can be solved by Gaussian elimination as follows:

In [6]:
I = np.identity(3)

**Iteration k = 1:**

Notice that, in this case, the pivot of the next Gauss transform is `2`. The pivot is not a small number. However, let's apply the partial pivoting for illustrating the procedure and showing that it does not change the final result.

In this case, we interchange the first and second rows/elements of `A0`/`y0`. This is equivalent to premultiply `A0` and `y0` by the following matrix:

In [7]:
P1 = np.identity(3)[[1,0,2]]

print P1

[[ 0.  1.  0.]
 [ 1.  0.  0.]
 [ 0.  0.  1.]]


In [8]:
print np.dot(P1, A)

[[-3. -1.  2.]
 [ 2.  1. -1.]
 [-2.  1.  2.]]


In [9]:
print np.dot(P1, y)

[-11.   8.  -3.]


Notice that this row permutation changed the pivot from `2` to `-3`.

In [10]:
u = np.array([1., 0., 0.])

In [11]:
t = np.array([0., np.dot(P1, A)[1][0]/np.dot(P1, A)[0][0], np.dot(P1, A)[2][0]/np.dot(P1, A)[0][0]])

In [12]:
A1 = (I - np.outer(t, u)).dot(np.dot(P1,A))

In [13]:
A1

array([[-3.        , -1.        ,  2.        ],
       [ 0.        ,  0.33333333,  0.33333333],
       [ 0.        ,  1.66666667,  0.66666667]])

In [14]:
y1 = (I - np.outer(t, u)).dot(np.dot(P1,y))

In [15]:
y1

array([-11.        ,   0.66666667,   4.33333333])

**Iteration k = 2:**

Now, we interchange the second and third rows/elements of `A1`/`y1`. This is equivalent to premultiply `A1` and `y1` by the following matrix:

In [16]:
P2 = np.identity(3)[[0,2,1]]

print P2

[[ 1.  0.  0.]
 [ 0.  0.  1.]
 [ 0.  1.  0.]]


In [17]:
print np.dot(P2, A1)

[[-3.         -1.          2.        ]
 [ 0.          1.66666667  0.66666667]
 [ 0.          0.33333333  0.33333333]]


In [18]:
print np.dot(P2, y1)

[-11.           4.33333333   0.66666667]


In [19]:
u = np.array([0., 1., 0.])

In [20]:
t = np.array([0., 0., np.dot(P2, A1)[2][1]/np.dot(P2, A1)[1][1]])

In [21]:
B = (I - np.outer(t, u)).dot(np.dot(P2, A1))

In [22]:
B

array([[-3.        , -1.        ,  2.        ],
       [ 0.        ,  1.66666667,  0.66666667],
       [ 0.        ,  0.        ,  0.2       ]])

In [23]:
z = (I - np.outer(t, u)).dot(np.dot(P2, y1))

In [24]:
z

array([-11.        ,   4.33333333,  -0.2       ])

Solution of this equivalent triangular system:

In [25]:
print np.linalg.solve(B,z)

[ 2.  3. -1.]


Solution of the original system:

In [26]:
print np.linalg.solve(A,y)

[ 2.  3. -1.]


## Algorithm implementation

Now, our equivalent triangular system was iteratively calculated according to the following algorithm:

$$
\begin{array}{ccccc}
\mathbf{A}^{(0)} = \mathbf{A} & & & \mathbf{y}^{(0)} = \mathbf{y} \\\\
\mathbf{A}^{(1)} = \left(\mathbf{I} - \mathbf{M}^{(1)}\right) \mathbf{P}^{(1)}\mathbf{A}^{(0)} & & &
\mathbf{y}^{(1)} = \left(\mathbf{I} - \mathbf{M}^{(1)}\right) \mathbf{P}^{(1)}\mathbf{y}^{(0)} \\\\
\mathbf{A}^{(2)} = \left(\mathbf{I} - \mathbf{M}^{(2)}\right) \mathbf{P}^{(2)}\mathbf{A}^{(1)} & & &
\mathbf{y}^{(2)} = \left(\mathbf{I} - \mathbf{M}^{(2)}\right) \mathbf{P}^{(2)}\mathbf{y}^{(1)}
\end{array} \: ,$$

where $\mathbf{P}^{(k)}$ is the permutation matrix used to interchange the rows and perform the partial pivoting. Or, alternatively,

$$
\mathbf{C}^{(0)} = \left[ \: \mathbf{A} \: \vert \: \mathbf{y} \: \right] \: ,
$$

$$
\begin{array}{c}
\mathbf{C}^{(1)} = \left(\mathbf{I} - \mathbf{M}^{(1)}\right) \mathbf{P}^{(1)}\mathbf{C}^{(0)} \\\\
\mathbf{C}^{(2)} = \left(\mathbf{I} - \mathbf{M}^{(2)}\right) \mathbf{P}^{(2)}\mathbf{C}^{(1)} 
\end{array} \: ,$$

where $\mathbf{B} = \mathbf{C}^{(2)}[ \, : \, , \, :N]$ and $\mathbf{z} = \mathbf{C}^{(2)}[ \, : \, , \, N+1]$.

Notice that a matrix $\mathbf{P}^{(k)}$ may interchange the set of rows $\left[ \, k \, , \, : \, \right]$. For example, while the matrix $\mathbf{P}^{(1)}$ can interchange all the rows forming the matrix $\mathbf{C}^{(0)}$, the matrix $\mathbf{P}^{(2)}$ can interchange only the set of rows $\left[ \, 2 \, , \, : \, \right]$ (from the second on) forming the matrix $\mathbf{C}^{(1)}$ .

For convenience, let's define $\tilde{\mathbf{C}}^{(k-1)} = \mathbf{P}^{(k)} \mathbf{C}^{(k-1)}$. Then, the computation of  $\mathbf{C}^{(k)}$ can be rewritten as follows:

$$
\begin{split}
\mathbf{C}^{(k)} &= \left( \mathbf{I} - \mathbf{M}^{(k)} \right) \tilde{\mathbf{C}}^{(k-1)} \\\\
&= \tilde{\mathbf{C}}^{(k-1)} - \mathbf{M}^{(k)}\tilde{\mathbf{C}}^{(k-1)} \\\\
&= \tilde{\mathbf{C}}^{(k-1)} - \mathbf{t}^{(k)} \otimes \left(\mathbf{u}^{(k)}\right)^{\top}\tilde{\mathbf{C}}^{(k-1)} \\\\
&= \tilde{\mathbf{C}}^{(k-1)} - \mathbf{t}^{(k)} \otimes \tilde{\mathbf{C}}^{(k-1)}
\left[ \, k \, , \, : \, \right]
\end{split}
$$

We know that the subtraction of the outer product $\mathbf{t}^{(k)} \otimes \tilde{\mathbf{C}}^{(k-1)}\left[ \, k \, , \, : \, \right]$ from the matrix $\mathbf{C}^{(k-1)}$ produces $\mathbf{C}^{(k)}\left[ \, i+1 \, , \, i \, \right] = 0$. Besides, 
notice that $\mathbf{t}^{(k)} \left[ \, : k\right] = 0$. Consequently, the outer product affects only the elements $\left[ \, k+1 : \, ,  \, k+1 : \, \right]$ of the matrix $\mathbf{C}^{(k)}$. Then, we may simplify the computations as follows:

$$
\mathbf{C}^{(k)} \left[ \, k+1 : \, ,  \, k+1 : \, \right] = 
\tilde{\mathbf{C}}^{(k-1)} \left[ \, k+1 : \, ,  \, k+1 : \, \right] - 
\mathbf{t}^{(k)} \left[ \, k+1 : \, \right] \otimes 
\tilde{\mathbf{C}}^{(k-1)}
\left[ \, k \, , \, k+1 : \, \right]
$$

Finally, instead of storing the Gauss multipliers in the Gauss vector $\mathbf{t}^{(k)} \left[ \, k+1 : \, \right]$, we may store them below the main diagonal of the matrix $\tilde{\mathbf{C}}^{(k-1)}$, e.g.,

$$
\tilde{\mathbf{C}}^{(k-1)} \left[ \, k+1 : \, ,  \, k \, \right] = 
\mathbf{t}^{(k)} \left[ \, k+1 : \, \right]
$$

and, consequently,

$$
\mathbf{C}^{(k)} \left[ \, k+1 : \, ,  \, k+1 : \, \right] = 
\tilde{\mathbf{C}}^{(k-1)} \left[ \, k+1 : \, ,  \, k+1 : \, \right] - 
\tilde{\mathbf{C}}^{(k-1)} \left[ \, k+1 : \, ,  \, k \, \right] \otimes 
\tilde{\mathbf{C}}^{(k-1)}
\left[ \, k \, , \, k+1 : \, \right]
$$

The Gaussian elimination with partial pivoting can be implemented as follows:

    N = y.size
    assert A.shape[1] == N, 'A columns must be equal to y size'
    C = np.vstack([A.T, y]).T
    
    for k = 1:N-1
        
        # permutation step (computation of C tilde)
        p, C = permut(C, k)
        
        # assert the pivot is nonzero
        assert C[k,k] != 0., 'null pivot!'
        
        # calculate the Gauss multipliers and store them 
        # in the lower part of C
        C[k+1:,k] = C[k+1:,k]/C[k,k]
        
        # zeroing of the elements in the ith column
        C[k+1:,k+1:] = C[k+1:,k+1:] - outer(C[k+1:,k], C[k,k+1:])
        
    return C[:,:N], C[:,N]

The permutation function can be defined as follows:

    permut (C, i):
        p = [j for j in range(C.shape[0])]
        imax = i + np.argmax(np.abs(C[i:,i]))
        if imax != i:
            p[i], p[imax] = p[imax], p[i]
        return p, C[p,:]

### Exercise

1) In your `my_functions.py` file, create a function that implement the algorithm shown above. Note that the algorithm shown above uses the function `permut`, receives the matrix `A` and the vector `y` and return two numpy arrays containing the equivalent triangular system, as well as the Gauss multipliers. 

2) Create a first test in your `test_my_functions.py` file. For this test, create a linear system and the associated equivalent triangular system (do not use the function implemented in item 1). Then use the function implemented in item 1 to compute an equivalent triangular system. Finally, compare the true and the computed triangular system. They must be equal to each other.

3) Create a second test in your `test_my_functions.py` file. In this test, create a matrix `A0` and a vector `x0` and use them to compute a vector `A0x0 = y0`. Then, use the function created in item 1 to compute the equivalent triangular system. Use one of your functions to compute a vector `x1` by solving the equivalent triangular system. Finally, compare the computed vector `x1` and the expected vector `x0`.

4) In your `test_my_functions.py` file, create a test for the function `permut` presented above. For this test, create a reference input and a reference output. Then, compare the result produced by the function `permut` and the reference output.

## Inverse matrices

Sometimes, we need to calculate the inverse of a matrix. The inverse of a $N \times N$ matrix $\mathbf{A}$ is commonly represented by $\mathbf{A}^{-1}$. The inverse satisfies the following property:

$$
\begin{split}
\mathbf{A}^{-1} \mathbf{A} &= \mathbf{I} \\
\mathbf{A} \mathbf{A}^{-1} &= \mathbf{I} 
\end{split} \: ,
$$

where $\mathbf{I}$ represents the identity matrix.

The second equation presented above can be conveniently rewritten by using a column partition given by:

$$
\mathbf{A} 
\left[ \mathbf{A}^{-1}\left[ \, : \, , \, 1  \right] \cdots \mathbf{A}^{-1} \left[ \, : \, , \, N  \right]  \right] = 
\left[ \mathbf{u}_{1} \cdots \mathbf{u}_{N} \right] \: ,
$$

where $\mathbf{u}_{i}$, $i = 1, \dots, N$, is a $N \times 1$ vector with all elements equal to zero, except the $i$th element, which is equal to $1$. The vectors $\mathbf{A}^{-1}\left[ \, : \, , \, i  \right]$ and $\mathbf{u}_{i}$ represent the $i$th column of $\mathbf{A}^{-1}$ and $\mathbf{I}$, respectively. This equation can then be separated into $N$ linear systems:

$$
\begin{split}
\mathbf{A} \, \mathbf{A}^{-1} \left[ \, : \, , \, 1 \right] &= \mathbf{u}_{1} \\
\mathbf{A} \, \mathbf{A}^{-1} \left[ \, : \, , \, 2 \right] &= \mathbf{u}_{2} \\
\vdots \\
\mathbf{A} \, \mathbf{A}^{-1} \left[ \, : \, , \, N \right] &= \mathbf{u}_{N}
\end{split} \: .
$$

This equation shows that each column of the inverse matrix $\mathbf{A}^{-1}$ can be calculated by solving a linear system.

### Exercise

1) In your `my_functions.py` file, create a function to compute the inverse of a matrix. The code must receive a matrix `A` and calculate its inverse `Ainv`, column by column, according to the scheme presented above.

2) Create two tests in your `test_my_functions.py` file. In the first test, create a matrix `A`, compute its inverse `Ainv` by using the function created in the previous item and verify if the products dot(A, Ainv) and dot(Ainv, A) are equal to the identity matrix. The second test must compare the inverse matrix computed by using your function and the inverse matrix computed by using the routine [`numpy.linalg.inv`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.inv.html). 